In [1]:
import tensorflow as tf
# 查询系统可用的 GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
# 确保有可用的 GPU 如果没有, 则会报错
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# 设置参数,该段务必在运行jupyter的第一段代码执行，否则会无法初始化成功
# 仅在需要时申请显存空间（程序初始运行时消耗很少的显存，随着程序的运行而动态申请显存）
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#### 一、有哪些计算图

- 静态计算图
- 动态计算图
- AutoGraph  结合了静态和动态的优势，将动态图转换为静态计算图，执行效率和编码效率兼顾

AutoGraph在tf2.0中通过 装饰器 @tf.function 实现功能

#### 二、AutoGraph使用规范

- 被 @tf.function 修饰的函数应尽量使用TersorFlow中的函数而不是Python中的其他函数
- 避免在 @tf.function 修饰的函数中内部定义tf.Variable
- 被 @tf.function 修饰的函数不可修改该函数外部的Python列表或字典等结构类型变量

1. 通过函数的对比来理解装饰器在tf 和 python 中的差异

In [2]:
import numpy as np

In [3]:
@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)

@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [4]:
# 两次重复执行都是一致的结果，这是通过静态图修饰后的结果
np_random()
np_random()

array([[-1.73459983,  1.61538514,  0.66146653],
       [ 1.86567705,  1.46774412, -0.37176511],
       [ 0.18410566, -0.30380434, -0.39712214]])
array([[-1.73459983,  1.61538514,  0.66146653],
       [ 1.86567705,  1.46774412, -0.37176511],
       [ 0.18410566, -0.30380434, -0.39712214]])


In [5]:
# 如果是随机定义不加装饰器是每次结果都不一致的
np.random.randn(3,3)

array([[ 0.48754128, -0.57609037, -0.99282892],
       [ 1.18290497,  0.78712168,  1.22719453],
       [ 0.37198268,  1.81922035,  0.32943382]])

In [6]:
np.random.randn(3,3)

array([[ 0.06239056, -0.47058712,  0.24050997],
       [ 0.27496017,  0.5153869 , -0.86236044],
       [-1.03141072, -0.82295432, -0.30087934]])

In [7]:
# 两次执行的结果是不一致的
tf_random()
tf_random()

[[-0.535547495 0.226212829 -2.96548247]
 [-0.378376365 -0.0536504053 0.57822907]
 [0.555427313 0.0683829188 0.975780249]]
[[0.33682844 0.310662597 -0.272838891]
 [0.225154296 1.09503865 -0.0793219954]
 [-0.322832167 0.350299239 0.0948889256]]


2. 避免在 @tf.function 修饰的函数内部定义 tf.Variable

In [114]:
x = tf.Variable(1.0, dtype = tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

In [115]:
outer_var()
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [116]:
@tf.function
def inner_var():
    x = tf.Variable(1.0, dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)

In [117]:
# 如果写在里面，那么程序会报错
inner_var()

ValueError: in user code:

    <ipython-input-116-1d8a7931fdd5>:3 inner_var  *
        x = tf.Variable(1.0, dtype = tf.float32)
    /home/hp/.local/lib/python3.8/site-packages/tensorflow/python/ops/variables.py:261 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    /home/hp/.local/lib/python3.8/site-packages/tensorflow/python/ops/variables.py:243 _variable_v2_call
        return previous_getter(
    /home/hp/.local/lib/python3.8/site-packages/tensorflow/python/ops/variables.py:66 getter
        return captured_getter(captured_previous, **kwargs)
    /home/hp/.local/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py:510 invalid_creator_scope
        raise ValueError(

    ValueError: tf.function-decorated function tried to create variables on non-first call.


3. 被 @tf.function 修饰的函数不可修改该函数外部的Python列表或字典等结构类型变量

In [118]:
tensor_list = []

def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [119]:
tensor_list = []

# 注意上面和下面的区别，下面的是不存在结果的
@tf.function
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]


#### 三、AutoGraph机制原理


当我们使用 @tf.function 装饰一个函数的时候，后面到底发生了什么？
- 创建计算图
- 执行计算图

In [9]:
@tf.function(autograph= True) # 创建计算图
def myadd(a,b):
    for i in tf.range(3):
        tf.print(i)
    c = a + b 
    print("tracing")
    return c

In [10]:
myadd(tf.constant('hello'),tf.constant('world'))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'helloworld'>

- 上面可以看到，python先调用标准输出流打印 tracing语句，然后看到第二个步骤的结果，TensorFlow
- 调用标准输出流打印1，2，3

In [11]:
myadd(tf.constant('good'),tf.constant('moring'))

0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'goodmoring'>

从这里可以看到，再次使用不同的输入参数调用@tf.function的时候，这一次只执行了第二步。不再
打印 'tracing'结果。

In [123]:
myadd(tf.constant(1),tf.constant(2))

tracing
0
1
2


<tf.Tensor: shape=(), dtype=int32, numpy=3>

当输入的参数是和之前的类型完全不同的时候，计算图又先执行了打印 'tracing'结果。
- 这是因为，输入参数的数据类型发生变化，已经创建的计算图不再适用。
- 我们需要重新创建计算图，执行计算图。

##### 需要注意如果调用被@tf.function装饰的函数时，输入的参数不是Tensor类型，则每次都会重新创建计算图。

In [124]:
# 会重复的调用
myadd('hello','world')
myadd('new','world')

tracing
0
1
2
tracing
0
1
2


<tf.Tensor: shape=(), dtype=string, numpy=b'newworld'>

#### 四、AutoGraph使用案例

In [3]:
x = tf.Variable(1.0,dtype=tf.float32)

In [4]:
x

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>

In [126]:
# 注意这里定义了数据类型是float，那么3一定要表示为3.0,否则报错，input_signature是定义张量签名
@tf.function(input_signature = [tf.TensorSpec(shape= [],dtype = tf.float32)])
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return (x)

In [127]:
add_print(tf.constant(3.0))

# add_print(tf.constant(3)) # 输入不符合张量签名的参数将报错

4


<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

In [128]:
# 利用tf.Module的子类化将其封装一下
class DemoModule(tf.Module):
    def __init__(self, init_value = tf.constant(0.0), name = None):
        super(DemoModule, self).__init__(name = name)
        with self.name_scope:  # 相当于 with.tf.name_scope('demo_module')
            self.x = tf.Variable(init_value, dtype = tf.float32, trainable = True)

    
    @tf.function(input_signature = [tf.TensorSpec(shape= [],dtype = tf.float32)])
    def addprint(self,a):
        with self.name_scope:
            self.x.assign_add(a)
            tf.print(self.x)
            return (self.x)

In [129]:
demo = DemoModule(init_value = tf.constant(1.0))
result = demo.addprint(tf.constant(5.0))

6


In [130]:
# 查看模块中的全部变量和全部可训练变量
print(demo.variables);print(demo.trainable_variables)


(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)
(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)


In [131]:
# 查看模块中的全部子模块
demo.submodules

()

In [132]:
# 模型的保存和加载，指定需要跨平台部署的方法
tf.saved_model.save(demo,'./data/',signatures= {'serving_defult':demo.addprint})

INFO:tensorflow:Assets written to: ./data/assets


In [133]:
demo2 = tf.saved_model.load('./data/')
demo2.addprint(tf.constant(5.0))

11


<tf.Tensor: shape=(), dtype=float32, numpy=11.0>

In [134]:
!saved_model_cli show --dir ./data/ --all # saved_model_cli提供了一种通过命令行检查并恢复模型的机制


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_defult']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['a'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: serving_defult_a:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: 'addprint'
    Option #1
      Callable with:

#### 更加复杂的一个全连接模型案例

In [147]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes = 10):
        super(MyModel, self).__init__(name = 'my_model')
        self.num_classes = num_classes

        self.dense_1 = tf.keras.layers.Dense(32, activation= 'relu')
        self.dense_2 = tf.keras.layers.Dense(num_classes)

    @tf.function(input_signature = [tf.TensorSpec([None,32],tf.float32)])
    def call(self,inputs):
        x = self.dense_1(inputs)
        return self.dense_2(x)

In [148]:
data = np.random.random((1000,32))
labels = np.random.random((1000,10))

optimizer = tf.keras.optimizers.SGD(learning_rate= 1e-3)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((data,labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [149]:
model = MyModel(num_classes = 10)

In [150]:
epochs = 3

In [151]:
for epoch in range(epochs):
    print('start of epoch %d' % (epoch,))

    # 遍历数据集的batch_size
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train)
            loss_value = loss_fn(y_batch_train,logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 200 == 0:
            print('train loos of step %s:%s' % (step, float(loss_value)))
            print('seen so far : %s samples' % ((step + 1) *64))

start of epoch 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

train loos of step 0:35.120452880859375
seen so far : 64 samples
start of epoch 1
train loos of step 0:26.85476303100586
seen so far : 64 samples
start of epoch 2
train loos of step 0:22.419435501098633
seen so far : 64 samples


In [146]:
tf.saved_model.save(model,'my_saved_model')

INFO:tensorflow:Assets written to: my_saved_model/assets
